In [ ]:
# 1. Kết nối Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Cài đặt Unsloth
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# 3. Cài đặt các thư viện phụ thuộc
!pip install "datasets<4.4.0" "trl<=0.24.0" peft jiwer

# 4. Kiểm tra Torch (Optional)
import torch

Mounted at /content/drive
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-roc0ghz0/unsloth_02885708e1a14de69b44f5732d9eff42
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-roc0ghz0/unsloth_02885708e1a14de69b44f5732d9eff42
  Resolved https://github.com/unslothai/unsloth.git to commit 2eb6b0d5f363a60ed3792ea1f04250537ac66939
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2

In [ ]:
import os
import json
from datasets import Dataset, concatenate_datasets

LINE_BASE = "/content/drive/MyDrive/OCR_Dataset_Line"
PARA_BASE = "/content/drive/MyDrive/OCR_Dataset_Paragraph"

def load_ocr_dataset(root_dir, dataset_type="Line"):
    """
    Hàm đọc dữ liệu đệ quy từ các folder con
    """
    conversations = []
    if not os.path.exists(root_dir):
        print(f"Không tìm thấy thư mục {root_dir}. Hãy kiểm tra lại!")
        return []

    # Lấy danh sách folder con (1, 2, ..., 250...)
    subfolders = sorted([f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))])

    print(f"Đang quét {dataset_type} tại {root_dir}: Tìm thấy {len(subfolders)} thư mục con.")

    count = 0
    for folder in subfolders:
        folder_path = os.path.join(root_dir, folder)
        label_file = os.path.join(folder_path, "label.json")

        if not os.path.exists(label_file):
            continue

        with open(label_file, 'r', encoding='utf-8') as f:
            try:
                labels = json.load(f)
            except:
                continue

        for img_name, text_label in labels.items():
            img_path = os.path.join(folder_path, img_name)
            if os.path.exists(img_path):
                instruction = "Hãy chuyển đổi hình ảnh văn bản viết tay này sang định dạng text tiếng Việt chính xác."

                conversation = [
                    {
                        "role": "user",
                        "content": [
                            {"type": "image", "image": img_path},
                            {"type": "text", "text": instruction}
                        ]
                    },
                    {
                        "role": "assistant",
                        "content": [
                            {"type": "text", "text": text_label}
                        ]
                    }
                ]
                conversations.append({"messages": conversation})
                count += 1
    return conversations

# 1. Load Tập Train (Line + Paragraph)
print("--- LOADING TRAIN DATA ---")
train_line = load_ocr_dataset(os.path.join(LINE_BASE, "train_data"), "Line Train")
train_para = load_ocr_dataset(os.path.join(PARA_BASE, "train_data"), "Para Train")

# Gộp và Xáo trộn
full_train_list = train_line + train_para
train_dataset = Dataset.from_list(full_train_list)
train_dataset = train_dataset.shuffle(seed=3407) # giúp model học xen kẽ

print(f"\nTỔNG DỮ LIỆU HUẤN LUYỆN: {len(train_dataset)} mẫu")
print(f"   (Line: {len(train_line)} | Paragraph: {len(train_para)})")

# 2. Load Tập Test (Giữ riêng để đánh giá)
print("\n--- LOADING TEST DATA ---")
test_line_list = load_ocr_dataset(os.path.join(LINE_BASE, "test_data"), "Line Test")
test_para_list = load_ocr_dataset(os.path.join(PARA_BASE, "test_data"), "Para Test")

# Lưu dạng list để dùng cho hàm evaluate sau này
print(f"Dữ liệu Test Line: {len(test_line_list)}")
print(f"Dữ liệu Test Para: {len(test_para_list)}")

--- LOADING TRAIN DATA ---
Đang quét Line Train tại /content/drive/MyDrive/OCR_Dataset_Line/train_data: Tìm thấy 249 thư mục con.
Đang quét Para Train tại /content/drive/MyDrive/OCR_Dataset_Paragraph/train_data: Tìm thấy 249 thư mục con.

TỔNG DỮ LIỆU HUẤN LUYỆN: 8141 mẫu
   (Line: 7028 | Paragraph: 1113)

--- LOADING TEST DATA ---
Đang quét Line Test tại /content/drive/MyDrive/OCR_Dataset_Line/test_data: Tìm thấy 6 thư mục con.
Đang quét Para Test tại /content/drive/MyDrive/OCR_Dataset_Paragraph/test_data: Tìm thấy 6 thư mục con.
Dữ liệu Test Line: 201
Dữ liệu Test Para: 31


In [ ]:
from unsloth import FastVisionModel
import torch

# 1. Load Model Qwen2-VL-7B-Instruct (4-bit Quantization)
model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit",
    load_in_4bit = True,
    use_gradient_checkpointing = "unsloth",
)

# 2. Cấu hình LoRA Adapter
# Target modules sẽ tự động được Unsloth tối ưu cho Qwen2-VL
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.8: Fast Qwen2_Vl patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.90G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

Unsloth: Making `model.base_model.model.model.visual` require gradients


In [ ]:
from transformers import Trainer, TrainingArguments
from PIL import Image
import torch

# 1. Data Collator (Hỗ trợ cả Line dài và Paragraph lớn)
def data_collator(examples):
    texts = []
    images = []

    for example in examples:
        msgs = example["messages"]
        img_path = msgs[0]["content"][0]["image"]
        instruction = msgs[0]["content"][1]["text"]
        ground_truth = msgs[1]["content"][0]["text"]

        try:
            image = Image.open(img_path).convert("RGB")
        except:
            continue

        prompt = f"<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>{instruction}<|im_end|>\n<|im_start|>assistant\n{ground_truth}<|im_end|>"
        texts.append(prompt)
        images.append(image)

    # 2. Xử lý Batch với Processor
    batch = tokenizer(
        text=texts,
        images=images,
        return_tensors="pt",
        padding=True,
        truncation=True,

        # --- CẤU HÌNH CHO PARAGRAPH ---
        # Tăng max_length để chứa đủ text của cả một đoạn văn
        max_length = 3072,

        # Giới hạn pixel an toàn cho T4 GPU.
        # Với ảnh paragraph to, nó sẽ downscale giữ tỉ lệ.
        # Với ảnh line nhỏ, nó giữ nguyên.
        max_pixels = 602112
    )

    # 3. Tạo Labels và Masking
    labels = batch["input_ids"].clone()
    if tokenizer.tokenizer.pad_token_id is not None:
        labels[labels == tokenizer.tokenizer.pad_token_id] = -100
    image_token_id = tokenizer.tokenizer.convert_tokens_to_ids("<|image_pad|>")
    if image_token_id is not None:
        labels[labels == image_token_id] = -100

    batch["labels"] = labels
    return batch

# 2. Training Arguments
training_args = TrainingArguments(
    per_device_train_batch_size = 1, # Bắt buộc là 1 vì Paragraph rất nặng VRAM
    gradient_accumulation_steps = 8, # Tích lũy gradient để mô phỏng batch lớn
    warmup_steps = 10,

    max_steps = 600,

    learning_rate = 2e-4,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = "outputs",
    report_to = "none",
    remove_unused_columns = False,
    eval_strategy = "no",
    save_strategy = "no",
    dataloader_num_workers = 0,
)

# 3. Trainer
trainer = Trainer(
    model = model,
    train_dataset = train_dataset,
    data_collator = data_collator,
    args = training_args,
    processing_class = tokenizer,
)

# 4. Start Training
print("Fine-tuning với Dataset Line + Paragraph")
try:
    trainer.train()
    print("Huấn luyện hoàn tất!")
except Exception as e:
    print(f"\nLỗi Training: {e}")

Fine-tuning với Dataset Line + Paragraph


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,141 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 50,855,936 of 8,342,231,552 (0.61% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.090200
2,2.111700
3,1.971800
4,2.443700
5,2.472800
6,1.717100
7,1.744800
8,1.782800
9,1.593500
10,1.671700


Huấn luyện hoàn tất!


In [ ]:
from jiwer import cer
from tqdm import tqdm
from PIL import Image

# Chuyển model sang chế độ Inference
FastVisionModel.for_inference(model)

def run_evaluation(test_data, dataset_name, max_samples=10):
    """
    Hàm đánh giá dùng chung cho cả 2 loại dataset
    """
    print(f"\nĐÁNH GIÁ TẬP: {dataset_name}:")
    predictions = []
    references = []

    # Chỉ lấy max_samples mẫu đầu tiên để test nhanh
    # Nếu muốn test hết, hãy bỏ [:max_samples]
    sample_subset = test_data[:max_samples]

    for item in tqdm(sample_subset):
        msgs = item["messages"]
        img_path = msgs[0]["content"][0]["image"]
        ground_truth = msgs[1]["content"][0]["text"]
        instruction = msgs[0]["content"][1]["text"]

        try:
            image = Image.open(img_path).convert("RGB")
        except:
            continue

        # Template input cho inference
        messages = [
            {"role": "user", "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": instruction}
            ]}
        ]

        # Tokenize & Generate
        input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
        inputs = tokenizer(
            image,
            input_text,
            add_special_tokens=False,
            return_tensors="pt",

            max_pixels=602112
        ).to("cuda")

        output = model.generate(**inputs, max_new_tokens=512) # Tăng max tokens output cho Paragraph
        decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

        # Tách kết quả
        if "assistant\n" in decoded_output:
            predicted_text = decoded_output.split("assistant\n")[-1].strip()
        else:
            predicted_text = decoded_output.strip()

        predictions.append(predicted_text)
        references.append(ground_truth)

    # Tính CER
    if len(references) > 0:
        score = cer(references, predictions)
        print(f"🎯 Kết quả {dataset_name} - CER: {score:.4f}")

        # In mẫu ví dụ
        print(f"--- Ví dụ mẫu ({dataset_name}) ---")
        print(f"Gốc : {references[0][:100]}...") # In 100 ký tự đầu
        print(f"Đoán: {predictions[0][:100]}...")
    else:
        print("Không có dữ liệu để đánh giá.")

# 1. Đánh giá Line-based
run_evaluation(test_line_list, "LINE DATASET", max_samples=10)

# 2. Đánh giá Paragraph-based
run_evaluation(test_para_list, "PARAGRAPH DATASET", max_samples=5)


ĐÁNH GIÁ TẬP: LINE DATASET:


100%|██████████| 10/10 [01:16<00:00,  7.68s/it]


🎯 Kết quả LINE DATASET - CER: 0.1904
--- Ví dụ mẫu (LINE DATASET) ---
Gốc : Thứ trưởng Bộ Tài nguyên & môi trường Đặng Hùng Võ tâm đắc với...
Đoán: Thủ trưởng Bộ Tài nguyên và môi trường Đặng Hùng Võ tâm đắc với...

ĐÁNH GIÁ TẬP: PARAGRAPH DATASET:


100%|██████████| 5/5 [02:06<00:00, 25.22s/it]

🎯 Kết quả PARAGRAPH DATASET - CER: 0.1392
--- Ví dụ mẫu (PARAGRAPH DATASET) ---
Gốc : Thứ trưởng Bộ Tài nguyên & môi trường Đặng Hùng Võ tâm đắc với cơ chế " khuyến khích nhà đầu tư thỏa...
Đoán: Thủ trưởng Bộ Tài nguyên và môi trường Đặng Hùng Võ tâm đắc với cơ chế "Khuyến khích nhà đầu tư thoá...


In [ ]:
from jiwer import cer
from tqdm import tqdm
from PIL import Image
import torch

# Chuyển model sang chế độ Inference (quan trọng để tắt dropout, tiết kiệm RAM)
FastVisionModel.for_inference(model)

def run_evaluation(test_data, dataset_name, max_eval_samples=20, num_print_samples=5):
    """
    Hàm đánh giá nâng cao:
    - max_eval_samples: Số lượng mẫu dùng để tính điểm CER (càng nhiều càng chính xác).
    - num_print_samples: Số lượng mẫu sẽ in ra màn hình để kiểm tra bằng mắt.
    """
    print(f"\n{'='*20}")
    print(f"ĐÁNH GIÁ TẬP: {dataset_name}")
    print(f"   (Tính toán trên {max_eval_samples} mẫu, Hiển thị {num_print_samples} mẫu)")
    print(f"{'='*20}")

    predictions = []
    references = []
    img_paths = [] # Lưu lại đường dẫn để dễ debug

    # Lấy subset dữ liệu để test
    sample_subset = test_data[:max_eval_samples]

    for item in tqdm(sample_subset):
        msgs = item["messages"]
        img_path = msgs[0]["content"][0]["image"]
        ground_truth = msgs[1]["content"][0]["text"]
        instruction = msgs[0]["content"][1]["text"]

        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Lỗi đọc ảnh {img_path}: {e}")
            continue

        # Template input
        messages = [
            {"role": "user", "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": instruction}
            ]}
        ]

        # Inference
        input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
        inputs = tokenizer(
            image,
            input_text,
            add_special_tokens=False,
            return_tensors="pt",
            max_pixels=602112 # Giữ nguyên setting như lúc train
        ).to("cuda")

        # Tăng max_new_tokens để không bị cắt giữa chừng với đoạn văn dài
        output = model.generate(**inputs, max_new_tokens=768)
        decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

        # Xử lý kết quả trả về
        if "assistant\n" in decoded_output:
            predicted_text = decoded_output.split("assistant\n")[-1].strip()
        else:
            predicted_text = decoded_output.strip()

        predictions.append(predicted_text)
        references.append(ground_truth)
        img_paths.append(img_path)

    # Tính CER tổng thể
    if len(references) > 0:
        score = cer(references, predictions)
        print(f"\nKẾT QUẢ - {dataset_name}")
        print(f"   Character Error Rate (CER): {score:.4f}")

        # In chi tiết các mẫu
        print("\n--- CHI TIẾT CÁC MẪU DỰ ĐOÁN ---")
        for i in range(min(num_print_samples, len(predictions))):
            print(f"\n[Mẫu số {i+1}]")
            print(f"Ảnh    : {img_paths[i]}")
            print(f"Gốc    : {references[i]}")
            print(f"Dự đoán: {predictions[i]}")

            # Đánh dấu nhanh đúng/sai (so sánh chuỗi đơn giản)
            if references[i] == predictions[i]:
                print("Hoàn toàn đúng")
            else:
                # Tính CER riêng cho mẫu này để xem mức độ sai
                sample_cer = cer(references[i], predictions[i])
                print(f"Có sai sót (CER: {sample_cer:.2f})")
            print("-" * 50)
    else:
        print("Không có dữ liệu hợp lệ để đánh giá.")

# --- THỰC THI ĐÁNH GIÁ ---

# 1. Đánh giá Line-based (Test 20 mẫu, in ra 5)
# Line ngắn nên chạy nhanh, bạn có thể tăng max_eval_samples lên 50 hoặc 100
run_evaluation(test_line_list, "LINE DATASET", max_eval_samples=20, num_print_samples=5)

# 2. Đánh giá Paragraph-based (Test 10 mẫu, in ra 3)
# Paragraph dài và nặng, nên test ít hơn để tiết kiệm thời gian chờ
run_evaluation(test_para_list, "PARAGRAPH DATASET", max_eval_samples=10, num_print_samples=3)


ĐÁNH GIÁ TẬP: LINE DATASET
   (Tính toán trên 20 mẫu, Hiển thị 5 mẫu)


100%|██████████| 20/20 [01:49<00:00,  5.47s/it]



KẾT QUẢ - LINE DATASET
   Character Error Rate (CER): 0.1465

--- CHI TIẾT CÁC MẪU DỰ ĐOÁN ---

[Mẫu số 1]
Ảnh    : /content/drive/MyDrive/OCR_Dataset_Line/test_data/250/1.jpg
Gốc    : Thứ trưởng Bộ Tài nguyên & môi trường Đặng Hùng Võ tâm đắc với
Dự đoán: Thủ trưởng Bộ Tài nguyên và môi trường Đặng Hùng Võ tâm đắc với
Có sai sót (CER: 0.05)
--------------------------------------------------

[Mẫu số 2]
Ảnh    : /content/drive/MyDrive/OCR_Dataset_Line/test_data/250/2.jpg
Gốc    : cơ chế " khuyến khích nhà đầu tư thỏa thuận với người sử dụng đất để nhận
Dự đoán: cố chế " Khuyến khích nhà đầu tư tham gia với người sử dụng đất để nhận
Có sai sót (CER: 0.12)
--------------------------------------------------

[Mẫu số 3]
Ảnh    : /content/drive/MyDrive/OCR_Dataset_Line/test_data/250/3.jpg
Gốc    : chuyển nhượng hoặc thuê quyền sử dụng đất, nhận góp vốn bằng quyền sử
Dự đoán: chuyển những khoản hoắc thuế quý giả sử dùng đất, nhận góp vốn bằng quyền sử?
Có sai sót (CER: 0.25)
---------------

100%|██████████| 10/10 [03:47<00:00, 22.75s/it]


KẾT QUẢ - PARAGRAPH DATASET
   Character Error Rate (CER): 0.1101

--- CHI TIẾT CÁC MẪU DỰ ĐOÁN ---

[Mẫu số 1]
Ảnh    : /content/drive/MyDrive/OCR_Dataset_Paragraph/test_data/250/1.jpg
Gốc    : Thứ trưởng Bộ Tài nguyên & môi trường Đặng Hùng Võ tâm đắc với cơ chế " khuyến khích nhà đầu tư thỏa thuận với người sử dụng đất để nhận chuyển nhượng hoặc thuê quyền sử dụng đất, nhận góp vốn bằng quyền sử dụng đất... " Nhưng tại hội nghị hôm qua, tiếng nói từ các sở tài nguyên & môi trường lại đều đồng thanh " loại bỏ " điều khoản này ra khỏi dự thảo. Giám đốc Sở Tài nguyên & môi trường Hải Phòng Chu Minh Tuấn cho biết thành phố cảng từng áp dụng cơ chế " thỏa thuận " trong một số rất ít trường hợp, vậy mà đã gây phản ứng theo kiểu dây chuyền khá mạnh : anh này nhận tiền rồi quay lại đòi nữa, anh kia thỏa thuận xong quay ra đòi thỏa thuận lại...
Dự đoán: Thủ trưởng Bộ Tài nguyên và môi trường Đặng Hùng Võ tâm đắc với cơ chế "Khuyến khích nhà đầu tư thoái thầu với người sử dụng đất để nhận ch